<a href="https://colab.research.google.com/github/pnnato/image_analysis/blob/master/maskrcnn_flower_tf_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Mask R-CNN instance segmentation with custom dataset in Google Colab
Jupyter notebook providing steps to train a **Matterport Mask R-CNN** model with custom dataset.

It runs in [Google Colab](https://colab.research.google.com/) using [Matterport framework](https://github.com/matterport/Mask_RCNN) with TensorFlow backend.

**Requirements are only dataset images and annotations file.**

**Colab Runtime type: Python3, GPU enabled.**

#Making Dataset
I generated dataset annotations with [VGG Image Annotator](http://www.robots.ox.ac.uk/~vgg/software/via/).

Notebook train a model for one class object detection. It is possible to slightly modify notebook to train model for multiple classes.

Before running notebook, we need to create dataset:


1.   Collect various pictures of objects to detect
3.   Create annotation files in VGG
4.   Create image.zip file having structure defined below
5.   Upload the zip file in your Google Drive

Zip file structure:
```
images.zip
|- "train" directory
  |- jpg image files of training data
  |- "via_region_data.json" annotations file of training data
|- "val" directory
  |- jpg image files of validation data
  |- "via_region_data.json" annotations file of validation data
```
Check my image.zip file as dataset example.

#Install required packages

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
%cd '/content/gdrive/My Drive'
  
!git clone --quiet https://github.com/matterport/Mask_RCNN.git

In [4]:
%cd '/content/gdrive/My Drive/Mask_RCNN'

!pip install -q PyDrive
!pip install -r requirements.txt
!python setup.py install

/content/gdrive/My Drive/Mask_RCNN
     |████████████████████████████████| 204kB 30.0MB/s 
     |████████████████████████████████| 163kB 48.4MB/s 
running install
running bdist_egg
running egg_info
writing mask_rcnn.egg-info/PKG-INFO
writing dependency_links to mask_rcnn.egg-info/dependency_links.txt
writing top-level names to mask_rcnn.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
writing manifest file 'mask_rcnn.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
copying mrcnn/utils.py -> build/lib/mrcnn
copying mrcnn/model.py -> build/lib/mrcnn
creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/model.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/visualize.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/parallel_model.py -> build/bdist.linux-x86_64/egg/mrcnn
copying build/lib/mrcnn/utils.py -> build/bdist.linux-x86_

#Download and extract dataset
Update fileId variable with Google Drive id of your image.zip dataset


In [5]:
# %cd ~/Mask_RCNN


# fileId = '1p11kagop07-LyNyTIQ5_bDHx6I2TSDN9'

# import os
# from zipfile import ZipFile
# from shutil import copy
# from pydrive.auth import GoogleAuth
# from pydrive.drive import GoogleDrive
# from google.colab import auth
# from oauth2client.client import GoogleCredentials

# os.makedirs('dataset')
# os.chdir('dataset')

# auth.authenticate_user()
# gauth = GoogleAuth()
# gauth.credentials = GoogleCredentials.get_application_default()
# drive = GoogleDrive(gauth)

# fileName = fileId + '.zip'
# downloaded = drive.CreateFile({'id': fileId})
# downloaded.GetContentFile(fileName)
# ds = ZipFile(fileName)
# ds.extractall()
# os.remove(fileName)
# print('Extracted zip file ' + fileName)

[Errno 2] No such file or directory: '/root/Mask_RCNN'
/content/gdrive/My Drive/Mask_RCNN


FileExistsError: ignored

#Edit settings file
*  find and replace occurrences of "balloon" and "Balloon" with name of your object
*  set epochs number


In [0]:
# %cd ~/Mask_RCNN

# !cp ~/Mask_RCNN/samples/balloon/balloon.py ./dog.py

# !sed -i -- 's/balloon/dog/g' dog.py
# !sed -i -- 's/Balloon/Dog/g' dog.py
# !sed -i -- 's/epochs=30/epochs=5/g' dog.py

/root/Mask_RCNN


In [0]:
%cd ~/Mask_RCNN

!cp ~/Mask_RCNN/samples/balloon/balloon.py ./flower.py

!sed -i -- 's/balloon/dog/g' flower.py
!sed -i -- 's/Balloon/Dog/g' flower.py
!sed -i -- 's/epochs=30/epochs=5/g' flower.py

/root/Mask_RCNN


In [0]:
%load_ext tensorboard

#Train model
Pretrained weights options are COCO, ImageNet or a model trained before

In [0]:
%cd '/content/gdrive/My Drive/Mask_RCNN'

!python flower3.py train --dataset=dataset/ --weights=last

/content/gdrive/My Drive/Mask_RCNN
Using TensorFlow backend.
Weights:  last
Dataset:  dataset/
Logs:  /content/gdrive/My Drive/Mask_RCNN/logs

Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_b

In [0]:
%tensorboard --logdir /logs/flower20200218T0041

#Run inference on test dataset

This part is evaluation of the trained model.

In [6]:
%cd '/content/gdrive/My Drive/Mask_RCNN'

import os
import cv2
import sys
import random
import math
import re
import time
import numpy as np
import tensorflow as tf
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import skimage
import glob
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
import mrcnn.model as modellib
from mrcnn.model import log

import flower3

# Root directory of the project
ROOT_DIR = os.getcwd()

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

custom_WEIGHTS_PATH = sorted(glob.glob(os.path.join(DEFAULT_LOGS_DIR, "*/mask_rcnn_*.h5")))[-1]
# print(custom_WEIGHTS_PATH)

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

config = flower3.FlowerConfig()
custom_DIR = os.path.join(ROOT_DIR, "dataset")

class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax
  
# Load validation dataset
dataset = flower3.FlowerDataset()
dataset.load_flower(custom_DIR, "val")

# Must call before using the dataset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

# load the last model you trained
# weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", custom_WEIGHTS_PATH)
model.load_weights(custom_WEIGHTS_PATH, by_name=True)

from importlib import reload # was constantly changin the visualization, so I decided to reload it instead of notebook
reload(visualize)

/content/gdrive/My Drive/Mask_RCNN


Using TensorFlow backend.



Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.9
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                15
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

<module 'mrcnn.visualize' from '/content/gdrive/My Drive/Mask_RCNN/mrcnn/visualize.py'>

In [9]:
#image_id = random.choice(dataset.image_ids)
APs = []
for image_id in dataset.image_ids:
  image, image_meta, gt_class_id, gt_bbox, gt_mask =\
      modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)
  info = dataset.image_info[image_id]
  print("image ID: {}.{} ({}) {}".format(info["source"], info["id"], image_id, 
                                         dataset.image_reference(image_id)))

  # Run object detection
  results = model.detect([image], verbose=1)

  #### Display results
  #ax = get_ax(1)
  #r = results[0]
  #visualize.display_instances(image, r['rois'], r['masks'], r['class_ids'], 
  #                            dataset.class_names, r['scores'], ax=ax,
  #                            title="Predictions")
  log("gt_class_id", gt_class_id)
  log("gt_bbox", gt_bbox)
  log("gt_mask", gt_mask)

  image, image_meta, gt_class_id, gt_bbox, gt_mask = modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

  molded_images = np.expand_dims(modellib.mold_image(image, config), 0)

  results = model.detect([image], verbose=0)
  r = results[0]

  AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'])
  APs.append(AP)

print("mAP: ", np.mean(APs))
  ## plot for single image only
  # AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'])
  # visualize.plot_precision_recall(AP, precisions, recalls)

image ID: flower.flower-102.jpg (0) /content/gdrive/My Drive/Mask_RCNN/dataset/val/flower-102.jpg
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1024, 1024, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 15)               min:    0.00000  max: 1024.00000  int64
anchors                  shape: (1, 261888, 4)        min:   -0.35390  max:    1.29134  float32
gt_class_id              shape: (4,)                  min:    1.00000  max:    1.00000  int32
gt_bbox                  shape: (4, 4)                min:  189.00000  max: 1010.00000  int32
gt_mask                  shape: (1024, 1024, 4)       min:    0.00000  max:    1.00000  bool
image ID: flower.flower-264.jpg (1) /content/gdrive/My Drive/Mask_RCNN/dataset/val/flower-264.jpg
Processing 1 images
image                    shape: (1024, 1024, 3)       min:    0.00000  max:  228.00000  uint8
molded_im

In [0]:
AP, precisions, recalls, overlaps = utils.compute_ap(gt_bbox, gt_class_id, gt_mask, r['rois'], r['class_ids'], r['scores'], r['masks'])
visualize.plot_precision_recall(AP, precisions, recalls)

In [0]:
%cd '/content/gdrive/My Drive/Mask_RCNN'

import tensorflow as tf

for event in tf.train.summary_iterator('logs/flower20200218T0744/events.out.tfevents.1582162177.6caa24d21757'):
  for value in event.summary.value:
    print(value.tag)
    if value.HasField('simple_value'):
      print(value.simple_value)

/content/gdrive/My Drive/Mask_RCNN
val_loss
0.5423080921173096
val_rpn_class_loss
0.003567649982869625
val_rpn_bbox_loss
0.13798962533473969
val_mrcnn_class_loss
0.055881109088659286
val_mrcnn_bbox_loss
0.1399851143360138
val_mrcnn_mask_loss
0.20488299429416656
loss
0.32387909293174744
rpn_class_loss
0.002845659153535962
rpn_bbox_loss
0.052371103316545486
mrcnn_class_loss
0.03564764931797981
mrcnn_bbox_loss
0.08061417192220688
mrcnn_mask_loss
0.15239891409873962
val_loss
0.4744507670402527
val_rpn_class_loss
0.002999836578965187
val_rpn_bbox_loss
0.11262751370668411
val_mrcnn_class_loss
0.05229530483484268
val_mrcnn_bbox_loss
0.11292827874422073
val_mrcnn_mask_loss
0.1935981959104538
loss
0.31455615162849426
rpn_class_loss
0.002828017110005021
rpn_bbox_loss
0.05196300521492958
mrcnn_class_loss
0.03816784545779228
mrcnn_bbox_loss
0.07039281725883484
mrcnn_mask_loss
0.15120285749435425
val_loss
0.4576110541820526
val_rpn_class_loss
0.0026529247406870127
val_rpn_bbox_loss
0.09908081591129